In [21]:
## Import all the things. 
import numpy as np
import csv
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from nltk.corpus import opinion_lexicon as op
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import pos_tag
np.set_printoptions(linewidth=400)   # optional: widens column of numpy array display

In [22]:
## get positve and negative words from the lexicon. 
pos_words = set(op.words('positive-words.txt'))
neg_words = set(op.words('negative-words.txt'))

In [23]:
## create lists for response and target. 
X = [] ; y = []
with open("reviews.txt", 'r', encoding='utf8') as reviews:
    for review in reviews:
        if review != "":
            review = review.strip()
            review = review.lower()
            review = word_tokenize(review)
            X.append(review[:-1])
            y.append(int(review[-1]))

In [24]:
## Open CSV file and record data line by line to read later. 

with open("reviews.csv", 'w', newline='') as rv:
    thefile = csv.writer(rv)
    thefile.writerow(['leng', 'rating'])
    for x,line in enumerate(X):
        posCount = 0 ; negCount = 0
        # create features of positive and negative words. 
        leng = len(line)
        for word in line:
            if word in pos_words:
                posCount +=1
            elif word in neg_words:
                negCount+=1
        # add a positive rating based on how much more pos reviews show over neg.
        one_star = 0; two_star = 0; three_star = 0; four_star=0; five_star = 0
        if negCount > (posCount*2):
            one_star = 1
        elif posCount > (negCount*2):
            five_star = 1
        elif negCount == posCount:
            three_star = 1
        elif negCount < posCount:
            four_star= 1
        elif posCount < negCount:
            two_star = 1
            

        #This is the tag
        rating = y[x]
        test_rating = y[x]
        line = [leng,one_star, two_star, three_star, four_star, five_star, rating]
        # write data to CSV file for later reading. 
        #print(line)
        thefile.writerow(line)
   
    

In [25]:
df = pd.read_csv("reviews.csv", header=0)
#df.shape

In [26]:
X = df.drop('rating', axis=1)
y = df['rating']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [28]:
for mclass in ('multinomial', 'ovr'):
    lr = LogisticRegression(solver='lbfgs', max_iter=30000, random_state=0, multi_class=mclass).fit(X_train, y_train)
    yhat = lr.predict(X_test)
     
    # the 3 lines below show how to invoke various output    
    print("\n",mclass,"Accuracy",accuracy_score(y_test, yhat))
    print("\n",mclass,"Classification Report\n",classification_report(y_test, yhat),sep="")
    print("\n",mclass,"Classification Report\n",confusion_matrix(y_test, yhat),sep="")


 multinomial Accuracy 0.688

multinomialClassification Report
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00        42
           4       0.00      0.00      0.00       102
           5       0.69      1.00      0.82       344

    accuracy                           0.69       500
   macro avg       0.14      0.20      0.16       500
weighted avg       0.47      0.69      0.56       500


multinomialClassification Report
[[  0   0   0   0   2]
 [  0   0   0   0  10]
 [  0   0   0   0  42]
 [  0   0   0   0 102]
 [  0   0   0   0 344]]


/home/loki/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 ovr Accuracy 0.688

ovrClassification Report
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00        42
           4       0.00      0.00      0.00       102
           5       0.69      1.00      0.82       344

    accuracy                           0.69       500
   macro avg       0.14      0.20      0.16       500
weighted avg       0.47      0.69      0.56       500


ovrClassification Report
[[  0   0   0   0   2]
 [  0   0   0   0  10]
 [  0   0   0   0  42]
 [  0   0   0   0 102]
 [  0   0   0   0 344]]


/home/loki/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
